In [ ]:
import random

# Distance Matrix

In [ ]:
dist_matrix = [
    [0, 31, 32, 5, 30, 9, 15, 40, 14, 21, 7, 13], 
    [31, 0, 5, 30, 40, 15, 8, 2, 4, 17, 50, 27], 
    [32, 5, 0, 32, 25, 16, 3, 3, 10, 8, 40, 21], 
    [5, 30, 32, 0, 50, 6, 30, 53, 10, 35, 12, 20], 
    [30, 40, 25, 50, 0, 32, 10, 20, 34, 7, 20, 6], 
    [9, 15, 16, 6, 32, 0, 15, 15, 4, 14, 21, 25], 
    [15, 8, 3, 30, 10, 15, 0, 6, 9, 4, 9, 9], 
    [40, 2, 3, 53, 20, 15, 6, 0, 7, 8, 30, 29], 
    [14, 4, 10, 10, 34, 4, 9, 7, 0, 13, 31, 35], 
    [21, 17, 8, 35, 7, 14, 4, 8, 13, 0, 12, 11], 
    [7, 50, 40, 12, 20, 21, 9, 30, 31, 12, 0, 5], 
    [13, 27, 21, 20, 6, 25, 9, 29, 35, 11, 5, 0]
]

num_cities = len(dist_matrix)

# Finding the TSP tour

In [ ]:
def tourLength(tour):
    dist = sum([dist_matrix[tour[i]][tour[i+1]] for i in range(len(tour)-1)])
    dist += dist_matrix[tour[-1]][tour[0]]
    return dist

Repeated nearest neighbour algorithm to generate a good initial tour

In [ ]:
def NN(start):
    tour = [start]
    while len(tour) < num_cities:
        nbs = [i for i in range(num_cities) if i not in tour]
        nxt = min(nbs, key=lambda x: dist_matrix[tour[-1]][x])
        tour.append(nxt)
        
    return tour, tourLength(tour)


def repeatedNN():
    tour, tourLength = NN(0)
    for i in range(1, num_cities):
        temp, tempLength = NN(i)
        if tourLength > tempLength:
            tour, tourLength = temp, tempLength
    
    return tour, tourLength

Simulated anneahling algorithm to improve upon the initial tour

In [ ]:
def SA(T_init=1000, repeats=7**6):
    e = 2.718281828459045
    current, currentCost = repeatedNN() 
    T = T_init
    for i in range(repeats):
        T = e**(-T/5)
        a, b = random.sample(range(num_cities), 2)
        current[a], current[b] = current[b], current[a]
        newCost = tourLength(current)
        if newCost < currentCost:
            currentCost = newCost
        else:
            x = random.uniform(0,1)
            if x < e**((currentCost - newCost)/T):
                currentCost = newCost
            else:
                current[a], current[b] = current[b], current[a]

    return current, currentCost

Finally, ant colony optimization algorithm to optimize the SA tour

In [ ]:
def AntColony(N=50, alpha=1, beta=2, p=0.1, numIter=500):
    best, bestLen = SA()
    T0 = 1/(num_cities*bestLen)
    T = [[T0 for i in range(num_cities)] for x in range(num_cities)]
    c = 2**-10
    for _ in range(numIter):
        antTours = []
        for node in range(N):
            visited = [random.randint(0, num_cities-1)]
            while len(visited) != num_cities:
                i = visited[-1]
                neighbours = [x for x in range(num_cities) if x not in visited]
                Tn = [(T[i][j]**alpha + c)*((1/(dist_matrix[i][j] + c))**beta) for j in neighbours]
                sigma_Tn = sum(Tn)
                p_ik = [t/sigma_Tn for t in Tn]
                j = random.choices(population=neighbours, k=1, weights=p_ik)
                visited += j
            
            # Record the ant's tour
            antTours.append((visited, tourLength(visited)))

        # Evaluate the tours made by ants
        TAnts = TBest = [[0 for i in range(num_cities)] for i in range(num_cities)]
        for tour, length in antTours:
            # Check if a better tour has been found, if so record it.
            if length < bestLen:
                best, bestLen = tour, length

            # Calculate the amount of pheromone to be deposited
            for i in range(num_cities-1):
                TAnts[tour[i]][tour[i+1]] += 1/length

            for i in range(num_cities-1):
                TBest[i][i+1] = 1/bestLen

        # Deposit and evaperate pheromone
        for i in range(num_cities):
            for j in range(num_cities):
                T[i][j] = max((1-p)*T[i][j] + TAnts[i][j] + TBest[i][j], T0)

    return best, bestLen

In [ ]:
AntColony()

([0, 3, 5, 8, 1, 7, 2, 6, 9, 4, 11, 10], 56)